## Exercise Set 3 for OSM 

### Dynamic Programming with John Stachurski

Exercises for the [OSM](https://bfi.uchicago.edu/osm) bootcamp dynamic programming section.

We will use the following libraries:

In [11]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt
from numba import jit

### Exercise 1.

Using Numba, as discussed in [this lecture](https://lectures.quantecon.org/py/need_for_speed.html), write your own version of NumPy's [interp](https://docs.scipy.org/doc/numpy/reference/generated/numpy.interp.html) function, specializing in linear interpolation in one dimension.  

Note that NumPy's function is compiled native machine code and hence is fast.  But try to beat if you can, at least in some scenarios, by using Numba to speed up your code.  Show a time comparison between the two functions, for some suitable choice of test.

### Exercise 2

Using your "Numbafied" linear interpolation function, try to use Numba to additionally speed up the endogenous grid method code from [this lecture](https://lectures.quantecon.org/py/egm_policy_iter.html).  Use CRRA utility and Cobb-Douglas production, as in that lecture, with the following parameter values.


Note: I didn't get much speed up.  I think because the outer loops don't matter much for speed, and hence it doesn't gain us much when we compile them.  

See how you go.

#### Exercise 1

In [99]:
def interp(vals, x_val, y_val):
    rv = np.zeros(len(vals))
    for k, val in enumerate(vals):
        for i in range(len(x_val)):
            if val >= x_val[i] and val <= x_val[i + 1]:
               rv[k]  = y_val[i] + (val - x_val[i]) * (y_val[i + 1] - y_val[i]) / (x_val[i + 1] - x_val[i])
    return rv

interp_nb = jit(interp)

x_val = list(np.linspace(0, 100, 10000))
y_val = list(np.linspace(20, -20, 10000))


In [101]:
interp_nb(np.array([3,2]), x_val, y_val)

array([ 18.8,  19.2])

In [102]:
%timeit interp_nb(np.array([3,2]),x_val, y_val)

1000 loops, best of 3: 389 µs per loop


In [105]:
%timeit np.interp(np.array([3,2]), x_val, y_val)

1000 loops, best of 3: 1.32 ms per loop


#### Exercise2

In [109]:
import numpy as np

def coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks, numb):
    """
    The approximate Coleman operator, updated using the endogenous grid
    method.  
    
    Parameters
    ----------
    g : function
        The current guess of the policy function
    k_grid : array_like(float, ndim=1)
        The set of *exogenous* grid points, for capital k = y - c
    beta : scalar
        The discount factor
    u_prime : function
        The derivative u'(c) of the utility function
    u_prime_inv : function
        The inverse of u' (which exists by assumption)
    f : function
        The production function f(k)
    f_prime : function
        The derivative f'(k)
    shocks : numpy array
        An array of draws from the shock, for Monte Carlo integration (to
        compute expectations).

    """

    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    if numb:
        Kg = lambda x: interp_nb(x, y, c)
    else:
        Kg = lambda x: np.interp(x, y, c)
    return Kg

In [110]:
## Define the model

alpha = 0.65
beta = 0.95
mu = 0
s = 0.1
grid_min = 1e-6
grid_max = 4
grid_size = 200
shock_size = 250

gamma = 1.5   # Preference parameter
gamma_inv = 1 / gamma

def f(k):
    return k**alpha

def f_prime(k):
    return alpha * k**(alpha - 1)

def u(c):
    return (c**(1 - gamma) - 1) / (1 - gamma)

def u_prime(c):
    return c**(-gamma)

def u_prime_inv(c):
    return c**(-gamma_inv)

def crra_coleman_egm(g, numb):
    return coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks, numb)


k_grid = np.linspace(grid_min, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))

In [114]:
## Iterate, compare policies

sim_length = 20

print("Timing policy function iteration with endogenous grid without numba")
g_init = lambda x: x
g = g_init
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm(g, False)
    g = new_g
qe.util.toc()


print("Timing policy function iteration with endogenous grid with numba")
crra_coleman_egm_numb = jit(crra_coleman_egm)
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm_numb(g, True)
    g = new_g
qe.util.toc()

Timing policy function iteration with endogenous grid without numba
TOC: Elapsed: 0.12627005577087402 seconds.
Timing policy function iteration with endogenous grid with numba
TOC: Elapsed: 0.3128190040588379 seconds.


0.3128190040588379

As we can see, the numbafied interpolation method performs better than the numpy interpolation method. But the endogenous grid point method doesn't benefit from using numba. As a matter of fact, the run time is longer for the numba function.